# WeedMap *Template Notebook*

Sam Blake, started 23 December 2022

### Import model and dependencies 

In [4]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook
%pylab notebook

import os
import glob
import numpy as np
import cv2

import matplotlib as mpl
from matplotlib import pyplot as plt

from tqdm import tqdm
from PIL import Image

import rasterio
from rasterio.plot import show

from aerial_imagery_classifier import imagery_statistical_binary_classification

plt.ioff()
plt.rcParams["figure.figsize"] = (10, 6)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


### Preprocessing

We assume the drone imagery has already had been preprocessed and split into manageable sized tiles. Preprocessing may include downsampling, Gaussian blurring and histogram equalisation. Use ```'downsample.py'``` and ```'equalise.py'``` for these steps. 

### Import weed and crop training samples

In [ ]:
crop_1 = cv2.imread('...')

In [ ]:
weed_1 = cv2.imread('...')

### Model calibration on single tiles

In [ ]:
filename = cv2.imread('...')

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [weed_1, weed_2, ...], \
            [crop_1, crop_2, ...], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (17,17), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = 64, \
            duplicate_tolerance = 5, \
            common_tolerance = 5, \
            n_sigma_thresholds = [2, 2.25, 2.5, 2,75, 3.], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 100, morph_close = 20, \
            dilation_size = 151, \
            z_score = True, \
            limit_outliers = False, \
            extents = [dataset.bounds.left, dataset.bounds.right, 
                       dataset.bounds.bottom, dataset.bounds.top], \
            export_histogram = False, \
            export_heatmap = False, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = True, \
            verbose = True)


# Export spray map to GeoTIFF.
output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, 255*spray_region)

# Export weed map to GeoTIFF.
output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

# Export classification CI to GeoTIFF.
output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, classification_confidence)

###  Model all tiles: 

In [ ]:
file_pattern = '..../tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue

    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [weed_1, weed_2, ...], \
            [crop_1, crop_2, ...], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (17,17), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = 64, \
            duplicate_tolerance = 5, \
            common_tolerance = 5, \
            n_sigma_thresholds = [2, 2.25, 2.5, 2,75, 3.], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 100, morph_close = 20, \
            dilation_size = 151, \
            z_score = True, \
            limit_outliers = False, \
            extents = [dataset.bounds.left, dataset.bounds.right, 
                       dataset.bounds.bottom, dataset.bounds.top], \
            export_histogram = False, \
            export_heatmap = False, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = True, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, 255*spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)